In [2]:
import pandas as pd

In [35]:
import json
from LLM_helper import llm
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.exceptions import OutputParserException 

def process_posts():
    df = pd.read_excel("./topic.xlsx")
    df = df[df["status"].isna()]
    text = df.iloc[0]["topic"]
    enriched_posts = complete_post(text)
    return enriched_posts

def complete_post(post):
    template = '''
    You are given a short LinkedIn post idea. Your task is to:
    1. Generate a well-written, complete LinkedIn post based on the idea.
    2. Extract up to two relevant tags based on the content of the post.
    
    Here is the idea for the LinkedIn post:  
    {post}
    '''

    pt = PromptTemplate.from_template(template)
    chain = pt | llm
    response = chain.invoke(input={"post": post})

    try:
        json_parser = JsonOutputParser()
        res = json_parser.parse(response.content)
    except OutputParserException:
        raise OutputParserException("Context too big. Unable to parse jobs.")
    
    # Assuming the LLM generates a well-formed post and tags
    raw_post = post
    generated_post = res.get('generated_post', '')
    tags = res.get('tags', [])

    return {
        "linkedin_post": generated_post,
        "llm_summary": generated_post,  # Using the generated post itself as the summary
        "tags": tags
    }


In [43]:
from ln_oauth import auth, headers

ModuleNotFoundError: No module named 'ln_oauth'

In [44]:
pip install ln_oauth

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ln_oauth (from versions: none)
ERROR: No matching distribution found for ln_oauth

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\melika\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import requests
import json

# Your LinkedIn access token
access_token = 'AQVsm2MdQfEX8rUFXzOXZKcQAheUZV3829Qo7pUns5FoCCgeItt9fRDHk30jv8yStMQQni6Qg-0WKbpA68FfoPj0GkSLxbQcpSuVtNjEpQEtRDmTJZPEtk_sV_iG2GbnFqGTw4QF8KuNgKdmmol68X0sIJq3u4JJb2Gpe4iZ2aS5pxC-s9cxPO77aGzNKU6pioI1Cc64RlBmV1aATpxkaiqgA_VHqlatIY2WpFA4VNhcqSWsd_cFxN_Q1X-DaDG8NehlS0HjlBWgko0qU8aqnddtT5V43dk69fvXslbUZMwK-kIeLmXrzxuw0sF3uLkMpRgwImLtrM7Drl-8sT0ULBBUFgZBNA'  # Replace with your actual access token

# LinkedIn API URL for creating a post
url = "https://api.linkedin.com/v2/ugcPosts"

# Set the headers, including the access token
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Create the post content
post_content = {
    "author": "urn:li:person:3LU-AXhw2I",  # Replace with your LinkedIn URN (you have the "sub" as part of your user ID)
    "lifecycleState": "PUBLISHED",
    "specificContent": {
        "com.linkedin.ugc.ShareContent": {
            "shareCommentary": {
                "text": "Hello World!"  # The content of your post
            },
            "shareMediaCategory": "NONE"
        }
    },
    "visibility": {
        "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
    }
}

# Send the request to create the post
response = requests.post(url, headers=headers, data=json.dumps(post_content))

# Check the response
if response.status_code == 201:
    print("Post created successfully!")
    print("Response:", response.json())
else:
    print(f"Failed to create post. Status code: {response.status_code}")
    print("Error:", response.text)


Error fetching organization data: 404, {"status":404,"code":"RESOURCE_NOT_FOUND","message":"No virtual resource found"}
